In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

sys.path.append("../core")
sys.path.append("../core/simulations")

import simulation_library, potentials, integrators, data_logging, visuals

from data import gen_double_moon_samples
from losses import getLoss
from trainer import Trainer
from optimizers import getOpt
from network_base import RealNVP
from network_logging import *

In [3]:
system_builder = simulation_library.SystemFactory()
system = simulation_library.System(dim = 2)
wca = potentials.WCAPotential(1, 1)
system.central_potential = potentials.DoubleWellPotential(a = 1,
                                                          b = 6,
                                                          c = 1,
                                                          d = 1)
system.add_particle(simulation_library.Particle(wca, np.array([0,1])))
system.get_integrator("metropolis", None, temp = 1)
coords_logger = data_logging.CoordinateLogger(system, 100)
energy_logger = data_logging.EnergyLogger(system, 100)
system.registerObserver(coords_logger)
system.registerObserver(energy_logger)

In [4]:
system.run(100000)

In [6]:
coords = np.array(coords_logger.coordinates, dtype=np.float32).squeeze()

In [7]:
loss = getLoss().basic_loss()
opt = getOpt().adam()
model = LogTargetPlot(LogGaussPlot(LogLoss(RealNVP(loss, opt, model_name='without_energy'))))
trainer = Trainer(model, coords)
trainer.train(20)